<a href="https://colab.research.google.com/github/luannapm/Projeto-SoulCode/blob/main/Escolas_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PYSPARK

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Instalação dos pacotes

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 13.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=02ff54070ed35513cb6c53e81d0df8a7d94fb4f56306bd0df493bd8900eeeaf9
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
pip install fsspec

     |████████████████████████████████| 132 kB 24.5 MB/s 


In [ ]:
pip install gcsfs

     |████████████████████████████████| 1.1 MB 34.9 MB/s 
     |████████████████████████████████| 160 kB 56.5 MB/s 
     |████████████████████████████████| 271 kB 73.4 MB/s 
     |████████████████████████████████| 192 kB 63.3 MB/s 


###Importação das bibliotecas 

In [ ]:
import pandas as pd
import pyspark.sql.functions as F
import os
import io
import gcsfs
from google.cloud import storage
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from io import BytesIO

###Configuração da SparkSession

In [ ]:
spark = (
SparkSession.builder\
        .master("local")\
        .appName("projeto_final_soul_code")\
        .config("spark.ui.port", "4050")\
        .getOrCreate()
) 
spark    

###Extração do CSV no Cloud Storage e criação de um novo dataframe PySpark





In [ ]:
#Variáveis de ambiente
json_key = '/content/drive/MyDrive/PROJETO-FINAL-GRUPO-04/NOTEBOOKS/APOIO/projetofinal-bc6-04-9731ef7087b5.json'
bucket_name = 'projetofinal-bc6-04'
file_name = '/Saída/escolas_2019_trat.csv'

In [ ]:
#Concedendo acesso para à Bucket a partir da chave Json
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_key

In [ ]:
#Armazenamento do caminho para o CSV
file_path = f'gs://{bucket_name}{file_name}'

In [ ]:
#Armazenamento do projeto GCP junto à chave de acesso JSON
fs = gcsfs.GCSFileSystem(project=bucket_name, token=json_key) 

FileNotFoundError: ignored

In [ ]:
#Abrindo o caminho do armazenamento e leitura do CSV com o Pandas
with fs.open(file_path): data = pd.read_csv(file_path, sep=',', encoding='UTF-8', header=0)

In [ ]:
#Criação de um novo dataframe em PySpark a partir do dataframe criado com o Pandas
df1 = spark.createDataFrame(data)
df1.show()

In [ ]:
df1.printSchema()

In [ ]:
#Convertendo os tipos de dados das novas colunas, para um tipo coerente
df1 = (df1.select(F.col('ID_ESCOLA').cast('int'),
                  F.col('SITUACAO_FUNCIONAMENTO').cast('int'),
                  F.col('ID_REGIAO').cast('int'),
                  F.col('ID_UF').cast('int'),
                  F.col('DEPENDENCIA_ADM').cast('int'),
                  F.col('LOCALIZACAO').cast('int'),
                  F.col('LOCALIZACAO_DIFERENCIADA').cast('int'),
                  F.col('AGUA_POTAVEL').cast('int'),
                  F.col('ENERGIA_REDE_PUBLICA').cast('int'),
                  F.col('ENERGIA_RENOVAVEL').cast('int'),
                  F.col('ENERGIA_INEXISTENTE').cast('int'),
                  F.col('ESGOTO_REDE_PUBLICA').cast('int'),
                  F.col('ESGOTO_INEXISTENTE').cast('int'),
                  F.col('LIXO_SERVICO_COLETA').cast('int'),
                  F.col('LABORATORIO_CIENCIAS').cast('int'),
                  F.col('LABORATORIO_INFORMATICA').cast('int'),
                  F.col('ACESSIBILIDADE_INEXISTENTE').cast('int'),
                  F.col('INTERNET_APRENDIZAGEM').cast('int'),
                  F.col('BANDA_LARGA').cast('int')
                  )
)
df1.printSchema()

In [ ]:
#Criação de lista da região Nordeste = 2 e Sudeste = 3
lista_regiao = [2, 3]

#Criação de um dataframe usando os filtrando de região
df_sud_nord = df1.filter(F.col('ID_REGIAO').isin(lista_regiao))

df_sud_nord.show(10)

In [ ]:
df_sud_nord.printSchema()

In [ ]:
#Criação de um dataframe da Região Nordeste == 2, UFs e situação de funcionamento apenas das escolas funcionando == 1
df_situacao = df_sud_nord.select(F.col("ID_REGIAO"), F.col("ID_UF"), F.col("SITUACAO_FUNCIONAMENTO"))\
                 .filter(F.col("ID_REGIAO").isin(lista_regiao))\
                 .filter(F.col("SITUACAO_FUNCIONAMENTO") == 1)

In [ ]:
#Criação de um dataframe das escolas em funcionamento por região.
df_sit_sum = df_situacao.groupBy(F.col("ID_REGIAO")).sum("SITUACAO_FUNCIONAMENTO")

In [ ]:
#Criação de um dataframe dos estados que possuem AGUA_POTAVEL, ENERGIA_REDE_PUBLICA, ESGOTO_REDE_PUBLICA nas escolas
df_acesso = (df_sud_nord.groupBy(F.col("ID_UF")).agg(
              F.sum(F.col("AGUA_POTAVEL")).alias("Soma_agua_potavel"),
              F.sum(F.col("ENERGIA_REDE_PUBLICA")).alias("Soma_energia"),
              F.sum(F.col("ESGOTO_REDE_PUBLICA")).alias("Soma_esgoto")
)).orderBy(F.col("ID_UF"))

In [ ]:
df_acesso.show()

In [ ]:
#Criação de um dataframe com uma nova coluna chamada COLETA_LIXO com a seguinte informação: Caso a escola tenha serviço de coleta de lixo, colocar o texto “Coleta de lixo”. Caso contrário, colocar o texto: “Não possui coleta de lixo”
df_coleta = (df_sud_nord.withColumn("COLETA_LIXO", F.when(F.col("LIXO_SERVICO_COLETA") == 1, "Coleta de lixo") 
                            .otherwise("Não possui coleta de lixo"))
)

df_coleta.select(F.col("ID_REGIAO"), F.col("ID_UF"), F.col("COLETA_LIXO")).show(20)

In [ ]:
#Criação de um dataframe com o ranking dos estados por localização 
df_ranking_localizacao = df_sud_nord.groupBy(F.col('ID_UF')).agg(
    F.sum(F.col("LOCALIZACAO")).alias("LOCALIZACAO")
)

w1 = Window.orderBy(F.col("LOCALIZACAO").desc())

df_ranking_localizacao = (df_ranking_localizacao.select(F.col("ID_UF"), F.col("LOCALIZACAO"))
    .withColumn("RANKING", F.row_number()
    .over(w1)))

df_ranking_localizacao.show(10)

In [ ]:
#Exibição do número de escolas por dependência administrativa: 1 - Federal, 2 - Estadual, 3 - Municipal, 4 - Privada
df_sud_nord.groupBy(F.col("DEPENDENCIA_ADM")).count().show()

###Carregamento para o bucket - UPLOAD

In [ ]:
data2 = df_sud_nord.toPandas()

In [ ]:
file_save = 'Saída/escolas_2019_sud_nord'
data2.to_csv(f'gs://{bucket_name}/{file_save}.csv', index=False)

#SPARKSQL

----

In [ ]:
#Criação de uma visão temporária local 
df_sud_nord.createOrReplaceTempView("tabela_escolas_2019")

In [ ]:
#1ª consulta tem a função de retornar todos os dados da tabela_escolas_2019
spark.sql("SELECT * FROM tabela_escolas_2019").show(10)

In [ ]:
#2.1ª consulta tem a função de mostrar o número de escolas da tabela_escolas_2019 agrupados por região Sudeste = 3 e Nordeste = 2
spark.sql("SELECT ID_REGIAO, COUNT(ID_REGIAO) AS Qtd_ESCOLAS FROM tabela_escolas_2019 GROUP BY ID_REGIAO").show()

In [ ]:
#2.2ª consulta tem a função de mostrar o número de escolas da tabela_escolas_2019 agrupados pelos estados da região Nordeste
spark.sql("SELECT ID_UF, COUNT(ID_UF) AS Qtd_ESCOLAS FROM tabela_escolas_2019 GROUP BY ID_UF ORDER BY ID_UF").show()

In [ ]:
#3.1ª Consulta tem a função de mostrar a quantidade de localização diferenciada das escolas na região Nordeste  da tabela_escolas_2019 agrupado pela localização diferenciada: 
spark.sql("SELECT LOCALIZACAO_DIFERENCIADA, COUNT(ID_REGIAO) AS Qtd_REGIAO_NORDESTE FROM tabela_escolas_2019 WHERE ID_REGIAO = 2 GROUP BY LOCALIZACAO_DIFERENCIADA").show()

In [ ]:
#3.2ª Consulta tem a função de mostrar a quantidade de localização diferenciada das escolas na região Sudeste  da tabela_escolas_2019 agrupado pela localização diferenciada: 
spark.sql("SELECT LOCALIZACAO_DIFERENCIADA, COUNT(ID_REGIAO) AS Qtd_REGIAO_SUDESTE FROM tabela_escolas_2019 WHERE ID_REGIAO = 3 GROUP BY LOCALIZACAO_DIFERENCIADA").show()

In [ ]:
#4.1ª Consulta tem a função de mostrar a quantidade de escolas por dependência administrativa da tabela_escolas_2019 agrupado pela região Nordeste
spark.sql("SELECT DEPENDENCIA_ADM, COUNT(ID_REGIAO) AS Qtd_REGIAO_NORDESTE FROM tabela_escolas_2019 WHERE ID_REGIAO = 2 GROUP BY DEPENDENCIA_ADM").show()

In [ ]:
#4.2ª Consulta tem a função de mostrar a quantidade de escolas por dependência administrativa da tabela_escolas_2019 agrupado pela região Sudeste
spark.sql("SELECT DEPENDENCIA_ADM, COUNT(ID_REGIAO) AS Qtd_REGIAO_SUDESTE FROM tabela_escolas_2019 WHERE ID_REGIAO = 3 GROUP BY DEPENDENCIA_ADM").show()

In [ ]:
#5.1ª Consulta tem a função de mostrar a quantidade de escolas localizadas na região Nordeste da tabela_escolas_2019 agrupado pela localização
spark.sql("SELECT LOCALIZACAO, COUNT(ID_REGIAO) AS Qtd_REGIAO_Nordeste FROM tabela_escolas_2019 WHERE ID_REGIAO = 2 GROUP BY LOCALIZACAO").show()

In [ ]:
#5.2ª Consulta tem a função de mostrar a quantidade de escolas localizadas na região Sudeste da tabela_escolas_2019 agrupado pela localização
spark.sql("SELECT LOCALIZACAO, COUNT(ID_REGIAO) AS Qtd_REGIAO_SUDESTE FROM tabela_escolas_2019 WHERE ID_REGIAO = 3 GROUP BY LOCALIZACAO").show()

In [ ]:
#5.3ª Consulta tem a função de mostrar a quantidade de escolas localizadas na zona rural  da tabela_escolas_2019 agrupado pelos estados do Nordeste e Sudeste
spark.sql("SELECT ID_UF, COUNT(LOCALIZACAO) AS Qtd_LOCALIZACAO_RURAL FROM tabela_escolas_2019 WHERE LOCALIZACAO = 2 GROUP BY ID_UF").show()

In [ ]:
#5.4ª Consulta tem a função de mostrar a quantidade de escolas localizadas na zona urbana da tabela_escolas_2019 agrupado pelos estados do Nordeste e Sudeste
spark.sql("SELECT ID_UF, COUNT(LOCALIZACAO) AS Qtd_LOCALIZACAO_URBANA FROM tabela_escolas_2019 WHERE LOCALIZACAO = 1 GROUP BY ID_UF").show()

In [ ]:
#6.1ª Consulta tem a função de mostrar a quantidade de escolas que possuem laboratório de ciências e laboratório de informática da tabela_escolas_2019 agrupado pelas região Nordeste e Sudeste
spark.sql("SELECT ID_REGIAO, SUM(LABORATORIO_CIENCIAS) AS Qtd_LABORATORIO_CIENCIAS, SUM(LABORATORIO_INFORMATICA) AS Qtd_LABORATORIO_INFORMATICA FROM tabela_escolas_2019 WHERE LABORATORIO_INFORMATICA = 1 OR LABORATORIO_CIENCIAS = 1 GROUP BY ID_REGIAO ORDER BY ID_REGIAO").show()

In [ ]:
#6.2ª Consulta tem a função de mostrar a quantidade de escolas que possuem laboratório de ciências e laboratório de informática da tabela_escolas_2019 agrupado pelos estados do Nordeste e Sudeste
spark.sql("SELECT ID_UF, SUM(LABORATORIO_CIENCIAS) AS Qtd_LABORATORIO_CIENCIAS, SUM(LABORATORIO_INFORMATICA) AS Qtd_LABORATORIO_INFORMATICA FROM tabela_escolas_2019 WHERE LABORATORIO_INFORMATICA = 1 OR LABORATORIO_CIENCIAS = 1 GROUP BY ID_UF ORDER BY ID_UF").show()

In [ ]:
#7ª Consulta tem a função de mostrar a quantidade de escolas que possuem laboratório de ciências e laboratório de informática da tabela_escolas_2019 agrupado pelos estados do Nordeste e Sudeste
spark.sql("SELECT ID_UF, SUM(INTERNET_APRENDIZAGEM) AS Qtd_INTERNET_APRENDIZAGEM, SUM(LABORATORIO_INFORMATICA) AS Qtd_LABORATORIO_INFORMATICA FROM tabela_escolas_2019 WHERE INTERNET_APRENDIZAGEM = 1 OR LABORATORIO_INFORMATICA =1 GROUP BY ID_UF ORDER BY ID_UF").show()

In [ ]:
#8ª Consulta tem a função de mostrar a quantidade de escolas que possuem energia elétrica da rede pública, energia renovável e energia inexistente da tabela_escolas_2019 agrupado pelos estados do Nordeste e Sudeste
spark.sql("SELECT ID_UF, SUM(ENERGIA_REDE_PUBLICA) AS Qtd_ENERGIA_REDE_PUBLICA, SUM(ENERGIA_RENOVAVEL) AS Qtd_ENERGIA_RENOVAVEL, SUM(ENERGIA_INEXISTENTE) AS Qtd_ENERGIA_INEXISTENTE FROM tabela_escolas_2019 WHERE ENERGIA_REDE_PUBLICA = 1 OR ENERGIA_RENOVAVEL = 1 OR ENERGIA_INEXISTENTE = 1 GROUP BY ID_UF ORDER BY ID_UF").show()

In [ ]:
#9ª Consulta tem a função de mostrar a falta de acessibilidade que não possuem nenhum tipo de acessibilidade como elevadores, corrimão, sinalização tátil e etc da tabela_escolas_2019 agrupado pelos estados do Nordeste e Sudeste
spark.sql("SELECT ID_UF, SUM(ACESSIBILIDADE_INEXISTENTE) AS Qtd_ACESSIBILIDADE_INEXISTENTE FROM tabela_escolas_2019 WHERE ACESSIBILIDADE_INEXISTENTE = 1  GROUP BY ID_UF ORDER BY ID_UF").show()

*INSIGHTS*


> A região Nordeste possui o maior número de escolas;

> Os estados da região Nordeste que possuem o maior número de escolas são: Maranhão e Bahia;

> Os estados da região Sudeste que possuem o maior número de escolas são: Minas Gerais e São Paulos;

> O maior número de escolas da região Nordeste estão localizadas na zona rural, com destaque para  estados de Bahia e Maranhão;

> O maior número de escolas da região Sudeste estão localizadas na zona urbana, com destaque para  estados de São Paulo e Minas Gerais;

> Na região Nordeste somente 2168 escolas estão localizadas em Assentamento, 1732 em Terra Indígena e 667 em Comunidades de Remanescentes Quilombolas.   

> Na região Sudeste somente 181 escolas estão localizadas em Assentamento, 292 em Terra Indígena e 73 em Comunidades de Remanescentes Quilombolas.

> O maior número de laboratórios de ciências e informática estão localizados na região Sudeste, com destaque para o estado de São Paulo. Já Sergipe é o estado com o menor número de laboratórios de ciências e informática.

> Quanto a correlação entre os indicadores Internet aprendizagem e Laboratórios de informática: São Paulo possui um maior número de escolas com internet aprendizagem do que com laboratórios de informática e o estado do Maranhão possui mais laboratórios de informática do que acesso a Internet.

> O estado de São Paulo possui o maior número de escolas com energia elétrica, o segundo estado com o maior número de escolas com energia renovável e também é um estado que não possui escolas sem energia elétrica;

> O estado da Bahia possui o maior número de escolas com energia elétrica e com energia renovável, mas também é o estado com o maior número de escolas sem energia elétrica;


> Os estados da Bahia e São Paulo possuem o maior número de escolas com falta de acessibilidade.






















